# Differential Evolution

Differential evolution is used for the solution of real-valued optimization problems. 

The search space is $\mathbb{R}^m$ for some $m \in \mathbb{N}$. The population is formed by $n$ individuals represented as follows:
$$
    \mathbf{x} = \left(x^1, \dots, x^m \right) \in \mathbb{R}^m
$$

## Differential mutation
Select three solution from the current population: $\mathbf{a}, \mathbf{b}, \mathbf{c}$.

Let's define the donor vector $\mathbf{d}$ as
$$
    \mathbf{d} = \mathbf{a} + F \cdot \left(\mathbf{b} - \mathbf{c} \right)
$$
where $F \in [0,2]$ is called mutation factor.

## Binomial crossover
Select one candidate solution $\mathbf{s}$ from the current population.
The solution $\mathbf{s}$ and the donor vector $\mathbf{d}$ are combined in a trial vector $\mathbf{t}$ which is defined as follows:
$$
    \mathbf{t}^j = 
    \begin{cases}
    \mathbf{d}^j & \text{if} \;\; rnd^j \leq p_{CR} \;\; \text{or} \;\; I_{rnd} == j \\
    \mathbf{s}^j & \text{otherwise}
    \end{cases}
$$

where $p_{CR}$ is the crossover probability, $I_{rnd} \in \{  1, \dots, m\}$ is a randomly selected index and $rnd$ are random numbers in $[0,1]$.

## DE selection
Given the parent $\mathbf{s}$ and the trial vector $\mathbf{t}_i$, selection is done by keeping only the individual with the best fitness:

- $\mathbf{s}$ is kept if it is fitter than $\mathbf{t}$;
- otherwise, $\mathbf{t}$ replaces $\mathbf{s}$

# DE on the Ackley function

The Ackley function in two dimensions is defined as:
$$
f(x, y) = -20\,\mathrm{exp}\left(-0.2 \sqrt{0.5(x^2 + y^2)}\right) - \mathrm{exp}\left( 0.5 (\cos(2\pi x) + \cos(2\pi y))\right) + e + 20
$$
the search space is $[-5, 5]^2$ and the global optimum is in $(0,0)$.

The function is characterized by a large number of local minima.

In [10]:
import numpy as np
import random
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact


class DifferentialEvolution():
    '''
    Differential Evolution algorithm for minimizing a function of several variables.
    Individual is represented as a vector of real numbers x=(x1, x2, ...).
    '''
    def __init__(self, pop_size, dim, bounds, fitness):
        self.pop_size = pop_size # number of individuals in the population
        self.dim = dim # dimension on the search space
        self.bounds = bounds # bound of the search space to be specified 
                             # as a list of tuples: [(min_x1, min_x2, ...), (max_x1, max_x2, ...)]
        self.fitness = fitness # function to be minimized 
        self.pop = None
        self.best_individual = None
        self.best_fitness = None
        self.history = []

    
    def save_3D_plot(self):
        '''
        Plot the population in 3D and save the image in img/de.
        '''
        if self.dim != 2:
            raise Exception("Can only plot 3D for dim = 2")
        
        ax = plt.figure(figsize=(4,4)).add_subplot(projection='3d')
        xvals = np.linspace(-5, 5, 201)
        yvals = np.linspace(-5, 5, 201)
        xx, yy = np.meshgrid(xvals, yvals)
        z = self.fitness([xx, yy])
        ax.plot_surface(xx, yy, z, antialiased=True, alpha=0.2)
        x_pop = [p[0] for p in self.pop]
        y_pop = [p[1] for p in self.pop]
        ax.scatter(x_pop, y_pop, c="red")
        plt.title("Generation " + str(len(self.history)))
        plt.savefig("img/de/generation_" + str(len(self.history)) + ".png")
        plt.close()


    def plot_generations(self):
        '''
        Plot a slider to visualize the evolution of the population.
        Each population is represented by the image saved in img/de.
        '''
        def print_results(i):
            plt.figure(figsize=(4,4))
            plt.imshow(plt.imread("img/de/generation_" + str(i) + ".png"))
            plt.show()
        interact(print_results, i=widgets.IntSlider(min=1,max=len(self.history),step=1,value=0))
        plt.show()

    
    def update_best(self):
        '''
        Update the best individual and its fitness
        '''
        self.best_individual = min(self.pop, key = self.fitness)
        self.best_fitness = self.fitness(self.best_individual)
        self.history.append([self.best_individual, self.best_fitness])


    def init_pop(self):
        '''
        Initialize the population with random individuals
        '''
        if self.pop is None:
            self.pop = []
            for i in range(self.pop_size):
                new_individual = np.random.uniform(self.bounds[0], self.bounds[1], self.dim)
                self.pop.append(new_individual)
            self.update_best()

    def mutation(self, F):
        '''
        Mutation is performed by randomly selecting three individuals from the population
        and creating a donor vector by adding the difference between two of them to the third.
        '''
        a,b,c = random.choices(self.pop, k=3)
        return a + F * (b - c)
    
    def crossover(self, solution, donor, p_cr):
        '''
        Crossover is performed by randomly selecting a position I_rnd in the donor vector
        and replacing the corresponding value in the solution vector with the value
        from the donor vector.
        For the other postions, the value from the donor vector is kept with probability p_cr.
        Otherwise, the value from the solution vector is kept.
        '''
        I_rnd = np.random.randint(0, self.dim-1)
        t = np.zeros(self.dim)
        for i in range(self.dim):
            if np.random.random() < p_cr:
                t[i] = donor[i]
            else:
                t[i] = solution[i]
        t[I_rnd] = donor[I_rnd]
        return t

    def selection(self, trial, solution):
        if self.fitness(trial) < self.fitness(solution):
            return trial
        else:
            return solution
    
    def run(self, F, p_cr, max_iter):
        '''
        For each iteration, the mutation/crossover/selection is 
        applied to each individual in the population creating a new population.
        '''
        for j in range(max_iter):
            self.save_3D_plot()
            next_gen = []
            for i in range(self.pop_size):
                individual = self.pop[i]
                donor = self.mutation(F)
                trial = self.crossover(individual, donor, p_cr)
                next_gen.append(self.selection(trial, individual))
            self.pop = next_gen
            self.update_best()

In [11]:
def ackley(x,y):
    return -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2))) - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))) + np.e + 20

In [12]:
DE = DifferentialEvolution(pop_size=100, dim=2, bounds=[[-5,-5], [5,5]], fitness= lambda individual : ackley(*individual))
DE.init_pop()
DE.run(F=1.3, p_cr=0.5, max_iter=50)

In [13]:
DE.plot_generations()

interactive(children=(IntSlider(value=1, description='i', max=51, min=1), Output()), _dom_classes=('widget-int…

<Figure size 400x400 with 0 Axes>

<Figure size 400x400 with 0 Axes>

<Figure size 400x400 with 0 Axes>

## Particle Swarm Optimization

First of all, we start by setting a series of parameters for PSO

In [14]:
c_soc = 1.49445
c_cog = 1.49445
w = 0.8

We can define how to update the position. Most of the code is to deal with the case when a particle goes outside the boundary of the search space. In this case the approach that we will take is the "dampened" one.

In [15]:
def update_position(position, velocity, boundaries):
    new_position = position + velocity
    for i, b in enumerate(boundaries):
        if new_position[i] < b[0]:
            new_position[i] = b[0]
            velocity[i] = - np.random.random() * velocity[i]
        elif new_position[i] > b[1]:
            new_position[i] = b[1]
            velocity[i] = - np.random.random() * velocity[i]
    return new_position

Updating the velocity can be done by splitting the update in the sum of three vectors, the social component, the cognitive component, and the inertia. The rest of the code only have to deal with keeping the velocities inside the allowed range of values.

In [16]:
def update_velocity(position, velocity, global_best, local_best, max_velocities):
    n = len(velocity)
    r1 = np.random.random(n)
    r2 = np.random.random(n)
    social_component = c_soc * r1 * (global_best - position)
    cognitive_component = c_cog * r2 * (local_best - position)
    inertia = w*velocity
    new_velocity = inertia + social_component + cognitive_component
    for i, v in enumerate(max_velocities):
        if np.abs(new_velocity[i]) < v[0]:
            new_velocity[i] = np.sign(new_velocity[i]) * v[0]
        elif np.abs(new_velocity[i]) > v[1]:
            new_velocity[i] = np.sign(new_velocity[i]) * v[1]
    return new_velocity

PSO can now be define. Differently from DE, in addition to the particle position we need:

- the particle velocity
- the local best for each particle
- the global best

The function returns, in addition to the global best, the history of the entire optimization process.

In [17]:
def pso(swarm_size, boundaries, max_velocities, n_iter, fit):
    m = len(boundaries)
    positions = [np.array([np.random.random() * (b[1] - b[0]) + b[0] for b in boundaries])
                 for i in range(0, swarm_size)]
    velocities = [np.array([np.random.choice([-1,1]) * np.random.uniform(v[0],v[1])
                            for v in max_velocities]) for i in range(0, swarm_size)]
    local_best = positions
    global_best = min(positions, key=fit)
    hist = [positions]
    for i in range(0, n_iter):
        velocities = [update_velocity(p, v, global_best, lb, max_velocities)
                      for p, v, lb in zip(positions, velocities, local_best)]
        positions = [update_position(p, v, boundaries) for p, v in zip(positions, velocities)]
        local_best = [min([p, lb], key=fit) for p, lb in zip(positions, local_best)]
        global_best = min([min(positions, key=fit), global_best], key=fit)
        hist.append(positions)
    return global_best, hist

We can now test PSO on the Ackley function:

In [18]:
np.random.seed(0)
best, hist = pso(10, [[-5, 5], [-5, 5]], [[0.001, 1], [0.001, 1]], 50, lambda p: ackley(*p))

In [19]:
plot_ackley(hist[50])

NameError: name 'plot_ackley' is not defined